<a href="https://colab.research.google.com/github/DmitryKutsev/ml_hw/blob/master/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz

--2020-03-01 07:44:51--  http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz
Resolving qwone.com (qwone.com)... 108.20.201.166
Connecting to qwone.com (qwone.com)|108.20.201.166|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17332201 (17M) [application/x-gzip]
Saving to: ‘20news-19997.tar.gz’

20news-19997.tar.gz 100%[===================>]  16.53M  6.88MB/s    in 2.4s    

2020-03-01 07:44:53 (6.88 MB/s) - ‘20news-19997.tar.gz’ saved [17332201/17332201]



In [0]:
!gunzip 20news-19997.tar.gz

In [3]:
!ls

20news-19997.tar  sample_data


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
count_vect = CountVectorizer()
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [5]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_dict = count_vect.fit(twenty_train.data) 
X_train_counts = X_train_dict.transform(twenty_train.data) 
X_train_counts.shape
X_train_counts

<2257x35788 sparse matrix of type '<class 'numpy.int64'>'
	with 365886 stored elements in Compressed Sparse Row format>

In [7]:
from sklearn.metrics import *
twenty_test = fetch_20newsgroups(subset='test', categories=categories, 
                                 shuffle=True, random_state=0)
X_test = count_vect.transform(twenty_test.data) 
print(X_test.shape)


(1502, 35788)


In [8]:
features = pd.DataFrame(X_train_counts)
answers = pd.DataFrame(twenty_train.target)
print(len(features))
len(answers)

2257


2257

In [0]:
def scores(model):
  ''' Функция для оценки моделей на тесте'''
  
  predicted = model.predict(X_test)
  acc = accuracy_score(predicted, twenty_test.target)
  micro_f1 = f1_score(predicted, twenty_test.target, average = 'micro')
  micro_p = precision_score(predicted, twenty_test.target, average = 'micro')
  micro_r = recall_score(predicted, twenty_test.target, average = 'micro')
  macro_f1 = f1_score(predicted, twenty_test.target, average = 'macro')
  macro_p = precision_score(predicted, twenty_test.target, average = 'macro')
  macro_r = recall_score(predicted, twenty_test.target, average = 'macro')
  print('acc={0:1.4f}'.format(acc))
  print('micro F1={0:1.4f}, micro P={1:1.4f}, micro R={2:1.4f}'.format(micro_f1, micro_p, micro_r))
  print('macro F1={0:1.4f}, macro P={1:1.4f}, macro R={2:1.4f}\n'.format(macro_f1, macro_p, macro_r))

In [0]:
folds = KFold(n_splits=n_fold, shuffle=True, random_state=0)
#кроссвалидация для Grid

In [23]:

parameter_grid = {'penalty' : ['l1', 'l2'],
                  'loss' : ['hinge', 'str', 'squared_hinge'],
                  'intercept_scaling' : [1, 2],
                  'verbose': [0,1]
                 }

grid_search = GridSearchCV(svc, param_grid=parameter_grid, cv=folds)
grid_search.fit(X_train_counts, twenty_train.target)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unsupported set of arguments: The combination of penalty='l1' and loss='hinge' is not supported, Parameters: penalty='l1', loss='hinge', dual=True

  FitFailedWarning)


[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unsupported set of arguments: loss='str' is not supported, Parameters: penalty='l1', loss='str', dual=True

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unsupported set of arguments: loss='str' is not supported, Parameters: penalty='l2', loss='str', dual=True

  FitFailedWarning)


[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unsupported set of arguments: The combination of penalty='l1' and loss='squared_hinge' are not supported when dual=True, Parameters: penalty='l1', loss='squared_hinge', dual=True

  FitFailedWarning)


[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Best score: 0.9539233038348083
Best parameters: {'intercept_scaling': 2, 'loss': 'squared_hinge', 'penalty': 'l2', 'verbose': 0}


In [0]:
chosen_svc = LinearSVC(intercept_scaling=2, loss='squared_hinge', penalty='l2', verbose=0).fit(X_train_counts, twenty_train.target)

In [25]:
scores(chosen_svc)

acc=0.8908
micro F1=0.8908, micro P=0.8908, micro R=0.8908
macro F1=0.8879, macro P=0.8866, macro R=0.8939



In [0]:
from sklearn import linear_model

In [27]:
lr = linear_model.LogisticRegression().fit(X_train_counts, twenty_train.target)

parameter_grid = {'class_weight' : ['balanced', None],
                  'penalty' : ['l2', 'l1'],
                  'solver' : ['liblinear', 'saga'],
                  'C' : [0.001, 0.01, 0.08],
                  'max_iter': [2,10]
                 }

grid_search = GridSearchCV(lr, param_grid=parameter_grid, cv=folds)
grid_search.fit(X_train_counts, twenty_train.target)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sa

Best score: 0.9570285152409046
Best parameters: {'C': 0.08, 'class_weight': 'balanced', 'max_iter': 10, 'penalty': 'l2', 'solver': 'liblinear'}


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [28]:
chosen_lr = linear_model.LogisticRegression(C=0.08, class_weight='balanced', max_iter=10, penalty='l2', \
                                            solver='liblinear').fit(X_train_counts, twenty_train.target)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [42]:
scores(chosen_lr)

acc=0.8955
micro F1=0.8955, micro P=0.8955, micro R=0.8955
macro F1=0.8927, macro P=0.8916, macro R=0.8995



In [0]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier().fit(X_train_counts, twenty_train.target)


In [31]:


parameter_grid = {'criterion' : ['gini', 'entropy'],
                  'splitter' : ['best', 'random'],
                  'min_samples_split' : [1, 2, 3],
                  'max_features' : ['auto', 'sqrt', 'log2'],
                  'ccp_alpha': [1.0, 2.0, 0.0]
                 }

grid_search = GridSearchCV(tree, param_grid=parameter_grid, cv=folds)
grid_search.fit(X_train_counts, twenty_train.target)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integ

Best score: 0.6641356932153392
Best parameters: {'ccp_alpha': 0.0, 'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_split': 3, 'splitter': 'random'}


In [0]:
chosen_tree = DecisionTreeClassifier(ccp_alpha=0.0, criterion='gini', max_features='sqrt', min_samples_split=3,\
                                     splitter='best').fit(X_train_counts, twenty_train.target)

In [33]:
scores(chosen_tree)

acc=0.5466
micro F1=0.5466, micro P=0.5466, micro R=0.5466
macro F1=0.5424, macro P=0.5444, macro R=0.5439



In [34]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [35]:
!pip install shap

     |████████████████████████████████| 276kB 2.6MB/s 
  Created wheel for shap: filename=shap-0.35.0-cp36-cp36m-linux_x86_64.whl size=394150 sha256=1a55f1c0ff00f7c116516819fab1464e23c896be1483a8824a8e09b875eae092
  Stored in directory: /root/.cache/pip/wheels/e7/f7/0f/b57055080cf8894906b3bd3616d2fc2bfd0b12d5161bcb24ac
Successfully built shap


In [36]:
import eli5
from eli5.sklearn import PermutationImportance
import shap

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
index_to_word = {v:k for k,v in X_train_dict.vocabulary_.items()}
def important_feachers(model, number_of_f):
  feacher_nums = eli5.formatters.as_dataframe.explain_weights_df(model, top=number_of_f)
  feacher_list = list(feacher_nums['feature'])
  my_list = []
  for i in feacher_list:
    if i != '<BIAS>':
      my_list.append(index_to_word[int(i[1:])])
  return(my_list)


In [48]:
for i in important_feachers(chosen_lr, 5):
  print(i)

keith
atheism
organization
subject
graphics
image
files
file
3d
pitt
doctor
msg
graphics
god
rutgers
church
posting
nntp
host


In [47]:
for i in important_feachers(chosen_tree, 5):
  print(i)

the
apr
kmr4
livesey
rushdie


In [46]:
for i in important_feachers(chosen_svc, 5):
  print(i)

mathew
okcforum
believing
atheism
graphics
package
images
library
card
pitt
doctor
information
graphics
rutgers
clh
christians
nntp
host


In [0]:
# Попробую добавить стоп-слова, возможно это почистит список признаков
count_vect = CountVectorizer(stop_words='english')
X_train_dict = count_vect.fit(twenty_train.data) 
X_train_counts = X_train_dict.transform(twenty_train.data) 
X_test = count_vect.transform(twenty_test.data)

In [52]:
chosen_svc1 = LinearSVC(intercept_scaling=2, loss='squared_hinge', penalty='l2', \
                        verbose=0).fit(X_train_counts, twenty_train.target)
scores(chosen_svc1)

acc=0.9001
micro F1=0.9001, micro P=0.9001, micro R=0.9001
macro F1=0.8973, macro P=0.8952, macro R=0.9038



In [53]:
chosen_tree1 = DecisionTreeClassifier(ccp_alpha=0.0, criterion='gini', max_features='sqrt', min_samples_split=3,\
                                     splitter='best').fit(X_train_counts, twenty_train.target)
scores(chosen_tree1)                                    

acc=0.5672
micro F1=0.5672, micro P=0.5672, micro R=0.5672
macro F1=0.5631, macro P=0.5662, macro R=0.5712



In [54]:
chosen_lr1 = linear_model.LogisticRegression(C=0.08, class_weight='balanced', max_iter=10, penalty='l2', \
                                            solver='liblinear').fit(X_train_counts, twenty_train.target)
scores(chosen_lr1)                                         

acc=0.9008
micro F1=0.9008, micro P=0.9008, micro R=0.9008
macro F1=0.8977, macro P=0.8960, macro R=0.9044



In [55]:
for i in important_feachers(chosen_lr, 5):
  print(i)

keith
atheism
organization
subject
graphics
image
files
file
3d
pitt
doctor
msg
graphics
god
rutgers
church
posting
nntp
host


In [0]:
# Если я правильно все понял, стало лучше. Попробую добавить биграммы.
count_vect = CountVectorizer(stop_words='english', ngram_range=(1,2))
X_train_dict = count_vect.fit(twenty_train.data) 
X_train_counts = X_train_dict.transform(twenty_train.data) 
X_test = count_vect.transform(twenty_test.data)

In [57]:
chosen_svc1 = LinearSVC(intercept_scaling=2, loss='squared_hinge', penalty='l2', \
                        verbose=0).fit(X_train_counts, twenty_train.target)
scores(chosen_svc1)

acc=0.9035
micro F1=0.9035, micro P=0.9035, micro R=0.9035
macro F1=0.9014, macro P=0.8991, macro R=0.9095



In [58]:
chosen_tree1 = DecisionTreeClassifier(ccp_alpha=0.0, criterion='gini', max_features='sqrt', min_samples_split=3,\
                                     splitter='best').fit(X_train_counts, twenty_train.target)
scores(chosen_tree1)  

acc=0.5999
micro F1=0.5999, micro P=0.5999, micro R=0.5999
macro F1=0.5916, macro P=0.5959, macro R=0.6022



In [59]:
chosen_lr1 = linear_model.LogisticRegression(C=0.08, class_weight='balanced', max_iter=10, penalty='l2', \
                                            solver='liblinear').fit(X_train_counts, twenty_train.target)
scores(chosen_lr1)   

acc=0.8981
micro F1=0.8981, micro P=0.8981, micro R=0.8981
macro F1=0.8957, macro P=0.8935, macro R=0.9052



In [61]:
for i in important_feachers(chosen_lr, 10):
  print(i)

keith
atheism
mathew
atheists
okcforum
rutgers
lines
organization
subject
graphics
image
files
file
3d
images
computer
software
god
people
pitt
doctor
msg
disease
information
treatment
christian
lines
graphics
god
rutgers
church
christians
christ
athos
christian
clh
posting
nntp
host


In [0]:
# Где-то хуже, где-то лучше. попробую поиграть с параметрами фильтрации признаков
count_vect = CountVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.5, min_df=2)
X_train_dict = count_vect.fit(twenty_train.data) 
X_train_counts = X_train_dict.transform(twenty_train.data) 
X_test = count_vect.transform(twenty_test.data)

In [63]:
chosen_svc1 = LinearSVC(intercept_scaling=2, loss='squared_hinge', penalty='l2', \
                        verbose=0).fit(X_train_counts, twenty_train.target)
scores(chosen_svc1)

acc=0.9041
micro F1=0.9041, micro P=0.9041, micro R=0.9041
macro F1=0.9018, macro P=0.8996, macro R=0.9095



In [64]:
chosen_tree1 = DecisionTreeClassifier(ccp_alpha=0.0, criterion='gini', max_features='sqrt', min_samples_split=3,\
                                     splitter='best').fit(X_train_counts, twenty_train.target)
scores(chosen_tree1)  

acc=0.6731
micro F1=0.6731, micro P=0.6731, micro R=0.6731
macro F1=0.6651, macro P=0.6706, macro R=0.6850



In [65]:
chosen_lr1 = linear_model.LogisticRegression(C=0.08, class_weight='balanced', max_iter=10, penalty='l2', \
                                            solver='liblinear').fit(X_train_counts, twenty_train.target)
scores(chosen_lr1) 

acc=0.9081
micro F1=0.9081, micro P=0.9081, micro R=0.9081
macro F1=0.9057, macro P=0.9038, macro R=0.9128



In [0]:
# Вроде опять лучше. Попробую увеличить еще.
count_vect = CountVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.6, min_df=3)
X_train_dict = count_vect.fit(twenty_train.data) 
X_train_counts = X_train_dict.transform(twenty_train.data) 
X_test = count_vect.transform(twenty_test.data)

In [67]:
chosen_svc1 = LinearSVC(intercept_scaling=2, loss='squared_hinge', penalty='l2', \
                        verbose=0).fit(X_train_counts, twenty_train.target)
scores(chosen_svc1)

acc=0.9008
micro F1=0.9008, micro P=0.9008, micro R=0.9008
macro F1=0.8989, macro P=0.8966, macro R=0.9061



In [68]:
chosen_tree1 = DecisionTreeClassifier(ccp_alpha=0.0, criterion='gini', max_features='sqrt', min_samples_split=3,\
                                     splitter='best').fit(X_train_counts, twenty_train.target)
scores(chosen_tree1) 

acc=0.7290
micro F1=0.7290, micro P=0.7290, micro R=0.7290
macro F1=0.7249, macro P=0.7282, macro R=0.7315



In [69]:
chosen_lr1 = linear_model.LogisticRegression(C=0.08, class_weight='balanced', max_iter=10, penalty='l2', \
                                            solver='liblinear').fit(X_train_counts, twenty_train.target)
scores(chosen_lr1) 

acc=0.9108
micro F1=0.9108, micro P=0.9108, micro R=0.9108
macro F1=0.9084, macro P=0.9063, macro R=0.9154



In [0]:
# И еще немного
count_vect = CountVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.7, min_df=4)
X_train_dict = count_vect.fit(twenty_train.data) 
X_train_counts = X_train_dict.transform(twenty_train.data) 
X_test = count_vect.transform(twenty_test.data)

In [71]:
chosen_svc1 = LinearSVC(intercept_scaling=2, loss='squared_hinge', penalty='l2', \
                        verbose=0).fit(X_train_counts, twenty_train.target)
scores(chosen_svc1)

acc=0.8988
micro F1=0.8988, micro P=0.8988, micro R=0.8988
macro F1=0.8969, macro P=0.8946, macro R=0.9040



In [72]:
chosen_tree1 = DecisionTreeClassifier(ccp_alpha=0.0, criterion='gini', max_features='sqrt', min_samples_split=3,\
                                     splitter='best').fit(X_train_counts, twenty_train.target)
scores(chosen_tree1) 

acc=0.6924
micro F1=0.6924, micro P=0.6924, micro R=0.6924
macro F1=0.6878, macro P=0.6886, macro R=0.7023



In [73]:
chosen_lr1 = linear_model.LogisticRegression(C=0.08, class_weight='balanced', max_iter=10, penalty='l2', \
                                            solver='liblinear').fit(X_train_counts, twenty_train.target)
scores(chosen_lr1) 

acc=0.9101
micro F1=0.9101, micro P=0.9101, micro R=0.9101
macro F1=0.9077, macro P=0.9057, macro R=0.9149



In [0]:
# Значения начали падать, значит оставлю так:
count_vect = CountVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.6, min_df=3)
X_train_dict = count_vect.fit(twenty_train.data) 
X_train_counts = X_train_dict.transform(twenty_train.data) 
X_test = count_vect.transform(twenty_test.data)

In [75]:
chosen_svc1 = LinearSVC(intercept_scaling=2, loss='squared_hinge', penalty='l2', \
                        verbose=0).fit(X_train_counts, twenty_train.target)
scores(chosen_svc1)

acc=0.9008
micro F1=0.9008, micro P=0.9008, micro R=0.9008
macro F1=0.8989, macro P=0.8966, macro R=0.9061



In [76]:
chosen_tree1 = DecisionTreeClassifier(ccp_alpha=0.0, criterion='gini', max_features='sqrt', min_samples_split=3,\
                                     splitter='best').fit(X_train_counts, twenty_train.target)
scores(chosen_tree1) 

acc=0.6498
micro F1=0.6498, micro P=0.6498, micro R=0.6498
macro F1=0.6404, macro P=0.6448, macro R=0.6566



In [77]:
chosen_tree1 = DecisionTreeClassifier(ccp_alpha=0.0, criterion='gini', max_features='sqrt', min_samples_split=3,\
                                     splitter='best').fit(X_train_counts, twenty_train.target)
scores(chosen_tree1)

acc=0.6658
micro F1=0.6658, micro P=0.6658, micro R=0.6658
macro F1=0.6667, macro P=0.6630, macro R=0.6745

